# Open a new session to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Inference with YOLOv5 from Hub


In [ ]:
def estimate_speed(location1, location2, time, distance_object, frame_width):
	mpp = distance_object * frame_width
	d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
	d_meters = d_pixels * mpp
	speed = d_meters / time
	return speed

In [ ]:
def get_centroid(x1,y1,x2,y2):
  c1 = float((x2 + x1) / 2.0)
  c2 = float((y2 + y1) / 2.0)
  return (c1, c2)

In [ ]:
import cv2
import torch
import time
import math
from PIL import Image
import numpy as np
import pandas as pd

print(torch.__version__)
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/gdrive/MyDrive/best28102020.pt', force_reload=True)  # local repo

str_video = '01067'

input_movie = cv2.VideoCapture('/content/gdrive/My Drive/your_directory/LIBRAS Data/'+ str_video +'.mp4')
imgs = []  # batched list of images
frame_number = 1
FRAME_WINDOW = 3
WIDTH_FRAME = 3#Get the width frame from the data read from opencv
DISTANCE_DEVICE_TO_PERSON = 1.2 # in meters (LIBRAS)

data_rows_list = []

#Lists to keep track of relevant data
speed_dominant_hand_list = []
speed_secondary_hand_list = []
centroid_secondary = (0,0)
centroid_dominant = (0,0)
speed_dominant = 0
speed_secondary = 0
position_dominant_hand_xs_list = []
position_dominant_hand_ys_list = []
position_secondary_hand_xs_list = []
position_secondary_hand_ys_list = []
frames = []

start = time.perf_counter()

while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    # height, width, number of channels in image
    VALID_FRAME = True
    try:
      height = frame.shape[0]
      WIDTH_FRAME = frame.shape[1]
      channels = frame.shape[2]
    except AttributeError:
      VALID_FRAME = False
    
    # Quit when the input video file ends
    if not ret:
        break

    if  frame_number % FRAME_WINDOW == 0 and VALID_FRAME == True:
      end = time.perf_counter()
      
      # Images
      image = frame[:, :, ::-1]
      # Inference
      imgs = [image]
      results = model(imgs, size=416)  # includes NMS
      #print(results.xyxy[0].cpu().numpy())
      # Results

      tensor = results.xyxy[0] 
      arr_np = tensor.cpu().numpy() 
      #print(f'Length arr_np: {len(arr_np)}')
      if len(arr_np) == 1:
        frames.append(frame_number)
        time_t = end - start
        #print(f'Elapse time: {time_t}')
        x1_1, y1_1, x2_1, y2_1 = arr_np[0][0], arr_np[0][1], arr_np[0][2], arr_np[0][3]
        #print(f'x1, y1, x2, y2: ({x1_1, y1_1}), ({x2_1, y2_1})')
        centroid_dominant = get_centroid(x1_1, y1_1, x2_1, y2_1)
        #print(f'centroid (c1,c2): {centroid[0], centroid[1]}')
        if len(position_dominant_hand_xs_list) == 0:
          speed_dominant = estimate_speed(centroid_dominant, centroid_dominant, time_t, DISTANCE_DEVICE_TO_PERSON, WIDTH_FRAME)
        else:
          speed_dominant = estimate_speed(centroid_dominant, (position_dominant_hand_xs_list[len(position_dominant_hand_xs_list) - 1], position_dominant_hand_ys_list[len(position_dominant_hand_ys_list) - 1]), time_t, DISTANCE_DEVICE_TO_PERSON, WIDTH_FRAME)
        position_dominant_hand_xs_list.append(centroid_dominant[0])
        position_dominant_hand_ys_list.append(centroid_dominant[1])
        speed_secondary = 0
        position_secondary_hand_xs_list.append(0)
        position_secondary_hand_ys_list.append(0)
        #speed_dominant_hand_list.append(speed)
        #print(f'estimated speed: {speed}')
      elif len(arr_np) == 2:
        frames.append(frame_number)
        time_t = end - start
        #print(f'Elapse time: {time_t}')
        x1_1, y1_1, x2_1, y2_1 = arr_np[0][0], arr_np[0][1], arr_np[0][2], arr_np[0][3]
        #print(f'x1_1, y1_1, x2_1, y2_1: ({x1_1, y1_1}), ({x2_1, y2_1})')
        centroid_dominant = get_centroid(x1_1, y1_1, x2_1, y2_1)
        #print(f'centroid (c1,c2): {centroid[0], centroid[1]}')
        if len(position_dominant_hand_xs_list) == 0:
          speed_dominant = estimate_speed(centroid_dominant, centroid_dominant, time_t, DISTANCE_DEVICE_TO_PERSON, WIDTH_FRAME)
        else:
          speed_dominant = estimate_speed(centroid_dominant, (position_dominant_hand_xs_list[len(position_dominant_hand_xs_list) - 1], position_dominant_hand_ys_list[len(position_dominant_hand_ys_list) - 1]), time_t, DISTANCE_DEVICE_TO_PERSON, WIDTH_FRAME)
        position_dominant_hand_xs_list.append(centroid_dominant[0])
        position_dominant_hand_ys_list.append(centroid_dominant[1])
        #speed_dominant_hand_list.append(speed)
        #print(f'estimated speed: {speed}')
        x1_2, y1_2, x2_2, y2_2 = arr_np[1][0], arr_np[1][1], arr_np[1][2], arr_np[1][3]
        centroid_secondary = get_centroid(x1_2, y1_2, x2_2, y2_2)
        if len(position_secondary_hand_xs_list) == 0:
          speed_secondary = estimate_speed(centroid_secondary, centroid_secondary, time_t, DISTANCE_DEVICE_TO_PERSON, WIDTH_FRAME)
        else:
          speed_secondary = estimate_speed(centroid_secondary, (position_secondary_hand_xs_list[len(position_secondary_hand_xs_list) - 1], position_secondary_hand_ys_list[len(position_secondary_hand_ys_list) - 1]), time_t, DISTANCE_DEVICE_TO_PERSON, WIDTH_FRAME)
        position_secondary_hand_xs_list.append(centroid_secondary[0])
        position_secondary_hand_ys_list.append(centroid_secondary[1])
        #print(f'x1_1, y1_1, x2_1, y2_1: ({x1_2, y1_2}), ({x2_2, y2_2})')

      start = time.perf_counter()

      data = [frame_number,centroid_dominant[0],centroid_dominant[1],speed_dominant,
              centroid_secondary[0],centroid_secondary[1],speed_secondary]

      data_rows_list.append(data)

    frame_number += 1

rows_list = []
for idx in range(len(data_rows_list)):
  rows_list.append('row_'+str(idx))

cols_list = []
for idx in range(len(data_rows_list[0])):
  cols_list.append('col_'+str(idx))

df1 = pd.DataFrame(data_rows_list,
                   index=rows_list)

df1.to_excel("/content/gdrive/My Drive/Doctorado/your_directory/LIBRAS Data/"+ str_video +"_hands_output_17102022.xlsx")  
